In [ ]:
# Libraries
import pandas as pd
import requests as rq
import re as re
import pickle

In [ ]:
# Get all links of the indicators
df_links = pd.read_excel("data_files/Data-sheet-STUDENTS.xlsx", sheet_name="Data sources", header=None, usecols="C", skiprows=3, na_values="")
df_links = df_links.dropna()
links = []
for index, link in df_links[2].items():
    # We don't handle other source for now
    if "data.worldbank" not in link:
        continue
        
    # Match the useful indicator in the link
    match = re.search("indicator/([\.|\w]*)", link)
    links.append(match.group(1))

In [ ]:
# Get countries data
df = pd.read_excel('data_files/Data-sheet-STUDENTS.xlsx', header=2, index_col=0)

In [ ]:
# Store the info of Country - ISO Code
iso_codes = {}

In [ ]:
# Get countries name for API request - Get ISO code
for index, name in df['Country (according to CN Gov):'].items():
    
    # Clean the data
    cleaned_name = name.split('(')[0].strip()
    if name == "Thailiand" :
        cleaned_name = "Thailand"
    if name == "The United Arab Emirates":
        cleaned_name = "United Arab Emirates"
    
    # We don't want to redo the request if the name already exists
    if name in iso_codes :
        continue
         
    # Request part
    apiURL = "https://restcountries.eu/rest/v2/name/" + cleaned_name
    resp = rq.get(apiURL)
        
    # Error handler
    if(resp.status_code != 200) :
        print("STATUS_CODE ERROR", resp.status_code, resp.text)
        continue
        
    # Response handler
    data = resp.json()
    alpha3Code = data[0]['alpha3Code']
    alpha2Code = data[0]['alpha2Code']
    if alpha3Code == "IOT":
        alpha3Code = "IND"
    iso_codes[name] = {"iso3" : alpha3Code, "iso2" : alpha2Code, "other" : cleaned_name}

In [ ]:
iso_codes

In [ ]:
# Save isocodes object for later use
with open('objects_saved/isocodes.pkl', 'wb') as handle:
  pickle.dump(iso_codes, handle)

In [ ]:
# Store the info of ISO Code + the other indicators
countries_inds = {}

In [ ]:
# Get all columns name of the indicators
for indic in links:
    for country in iso_codes:

        # If the data is already retrieved we don't make the request
        if country not in countries_inds.keys():
            countries_inds[country] = {}
        elif indic in countries_inds[country].keys():
            continue
        
        # Request part
        apiURL = "http://api.worldbank.org/v2/country/"+ iso_codes[country]['iso3'] +"/indicator/"+ indic +"?format=json&mrv=1"
        resp = rq.get(apiURL)

        # Error handler
        if(resp.status_code != 200) :
            print("STATUS_CODE ERROR", resp.status_code, resp.text)
            print("Indicator : " + indic + " - country : " + iso_codes[country]['iso3'])
            print("URL : " + apiURL)
            continue

        # Response handler
        # print("indicator : " + indic + " - country : " + country)
        data = resp.json()
        if data[1] is None:
            indic_value = 0
        else :
            indic_value = data[1][0]['value']
        countries_inds[country].update({indic : indic_value})

In [ ]:
countries_inds

In [ ]:
for country in countries_inds.keys():
    for indic in countries_inds[country]:
        if indic == "GC.TAX.IMPT.ZS":
            if pd.isnull( df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].iloc[list(countries_inds).index(country)]):
                df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
                # df.loc[list(countries_inds).index(country), 'IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = countries_inds[country][indic]
        elif indic == "FB.AST.NPER.ZS":
            if pd.isnull( df['WB IMF Non-performing loans to total gross loans ratio (%)'].iloc[list(countries_inds).index(country)]):
                df['WB IMF Non-performing loans to total gross loans ratio (%)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IS.RRS.GOOD.MT.K6":
            if pd.isnull( df['UIC Rail freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)]):
                df['UIC Rail freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IS.AIR.GOOD.MT.K1":
            if pd.isnull( df['ICAO air freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)]):
                df['ICAO air freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "NE.CON.PRVT.PP.CD":
            if pd.isnull( df['WB Household final consumption expenditure, PPP bn USD'].iloc[list(countries_inds).index(country)]):
                df['WB Household final consumption expenditure, PPP bn USD'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "GB.XPD.RSDV.GD.ZS":
            if pd.isnull( df['WB UNESCO R&D Expenditure % of GDP'].iloc[list(countries_inds).index(country)]):
                df['WB UNESCO R&D Expenditure % of GDP'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "SP.POP.SCIE.RD.P6":
            if pd.isnull( df['WB UNESCO Researchers p. mn inhabitants'].iloc[list(countries_inds).index(country)]):
                df['WB UNESCO Researchers p. mn inhabitants'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IC.REG.COST.PC.ZS":
            if pd.isnull( df['WB Cost of Starting a Business'].iloc[list(countries_inds).index(country)]):
                df['WB Cost of Starting a Business'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic] 
        elif indic == "GC.DOD.TOTL.GD.ZS":
            if pd.isnull( df['WB Central Government Debt'].iloc[list(countries_inds).index(country)]):
                df['WB Central Government Debt'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic] 
        elif indic == "SH.XPD.CHEX.PP.CD":
            if pd.isnull( df['WHO Current Health Expenditure p.c., PPP USD'].iloc[list(countries_inds).index(country)]):
                df['WHO Current Health Expenditure p.c., PPP USD'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IP.PAT.RESD":
            if pd.isnull( df['WIPO Patent Applications'].iloc[list(countries_inds).index(country)]):
                df['WIPO Patent Applications'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic] 
        elif indic == "IP.TMK.RSCT":
            if pd.isnull( df['WIPO Trademark applications'].iloc[list(countries_inds).index(country)]):
                df['WIPO Trademark applications'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "BX.KLT.DINV.WD.GD.ZS":
            if pd.isnull( df['IFDI Net Inflows (% of GDP)'].iloc[list(countries_inds).index(country)]):
                df['IFDI Net Inflows (% of GDP)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]   


In [ ]:
df

In [37]:
df.to_pickle("objects_saved/data.pkl")